# Teil 2: Transformieren, Extrahieren und Visualisieren von Features
## Mit Polars und Seaborn

Seaborn bietet uns eine einfache Möglichkeit um Grafiken übersichtlich und visuell ansprechend aufzubereiten, ohne uns in den komplexen Einstellungsmöglichkeiten von matplotlib zu verlieren, auf welcher seaborn basiert.  

In [977]:

import numpy as np
import scipy as sp
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme()

In [978]:

# read in the previously created parquet file
df = pl.read_parquet("resources/data/gta_v/gta_v_data.parquet")
print(df.head(3))

shape: (3, 36)
┌───────┬────────────┬────────────┬──────┬─────┬────────────┬────────────┬────────────┬────────────┐
│ title ┆ vehicle_cl ┆ manufactur ┆ feat ┆ ... ┆ resale_pri ┆ resale_pri ┆ top_speed_ ┆ top_speed_ │
│ ---   ┆ ass        ┆ er         ┆ ures ┆     ┆ ce_base    ┆ ce_upgrade ┆ real_kmh   ┆ in_game_km │
│ str   ┆ ---        ┆ ---        ┆ ---  ┆     ┆ ---        ┆ ---        ┆ ---        ┆ h          │
│       ┆ str        ┆ str        ┆ str  ┆     ┆ f64        ┆ f64        ┆ f64        ┆ ---        │
│       ┆            ┆            ┆      ┆     ┆            ┆            ┆            ┆ f64        │
╞═══════╪════════════╪════════════╪══════╪═════╪════════════╪════════════╪════════════╪════════════╡
│ Volat ┆ Planes     ┆ NA         ┆ Armo ┆ ... ┆ 2.2344e6   ┆ 2.421475e6 ┆ 266.35     ┆ 250.0      │
│ ol    ┆            ┆            ┆ red  ┆     ┆            ┆            ┆            ┆            │
│       ┆            ┆            ┆ Vehi ┆     ┆            ┆            ┆  

Bei Listen ist es etwas schwieriger, da die Spalten nicht einfach so aufgeteilt werden können, ohne die Gesamtanzahl stark zu erhöhen.
Wir erstellen _eine_ neue Spalte, welche nur die Anzahl der Features enthält (diese sind getrennt durch Kommata).
Damit können wir zwar nicht zwischen den Features unterscheiden, aber wir können zumindest statistisch Tendenzen feststellen.


In [979]:

df = df.with_columns(
        pl.col("features").str.split(',').arr.lengths().alias("feature_count"),
        # alternative mit apply einer python lambda funktion (langsamer)
        # pl.col("features").str.split(',').apply(lambda x: len(x)).alias("feature_count"),
) 
print(df.get_column("feature_count"))

shape: (567,)
Series: 'feature_count' [u32]
[
	8
	3
	9
	7
	1
	3
	4
	2
	2
	1
	1
	1
	...
	2
	1
	11
	4
	12
	3
	2
	2
	1
	1
	2
	1
	1
]
